# NiftyPET Example

This is a full demo of NiftyPET's default [OSEM](#OSEM "ordered subsets expectation maximisation") ($n_\text{max}=14$ subsets, span 11, Siemens Biograph mMR resolution), as well as a custom, explicit [MLEM](#MLEM "maximum likelihood expectation maximisation") incorporating [RM](#RM "resolution modelling").


Mathematically:

$$
\theta^{(k+1)} = {\theta^{(k)} \over \sum_n{H^TX_n^T1}}
    \circ
    \sum_n{ H^TX_n^T {m \over X_nH\theta^{(k)} + r + s} },
$$

- $k$ is iteration number
- $H$ applies a Gaussian PSF
- $X_n$ is the system matrix for subset $n$ (MLEM has just one subset)
- $m, r, s$ are measured, randoms, and scatter

## TODO

- $A$, $N$ in equation?
  + consistency with http://eknygos.lsmuni.lt/springer/370/63-91.pdf Eq. (45)

----

- Author: Casper O. da Costa-Luis [casper.dcl@{physics.org|ieee.org|kcl.ac.uk}](mailto:casper.dcl@physics.org)
- Date: June 2019

----

# Imports

In [ ]:
# imports
from __future__ import print_function, division
%matplotlib notebook

from niftypet import nipet
from niftypet import nimpa

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from tqdm.auto import trange
from brainweb import volshow

from os import path
import functools
import logging
logging.basicConfig(level=logging.INFO)

print(nipet.gpuinfo())
# get all the scanner parameters
mMRpars = nipet.get_mmrparams()
# conversion for Gaussian sigma/[voxel] to FWHM/[mm]
SIGMA2FWHMmm = (8 * np.log(2))**0.5 * np.array([mMRpars['Cnt']['SO_VX' + i] for i in 'ZYX']) * 10

# Load & Process Raw Data

In [ ]:
folderin = "amyloidPET_FBP_TP0"

# automatically categorise the input data
#logging.getLogger().setLevel(logging.INFO)
datain = nipet.classify_input(folderin, mMRpars)

# output path
opth = path.join(datain['corepath'], 'niftyout')
# switch on verbose mode
#logging.getLogger().setLevel(logging.DEBUG)

datain

In [ ]:
# hardware mu-map (bed, head/neck coils)
mu_h = nipet.hdw_mumap(datain, [1,2,4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# MR-based human mu-map

# UTE-based object mu-map aligned (need UTE sequence or T1 for pseudo-CT)
#mu_o = nipet.align_mumap(
#    datain,
#    scanner_params=mMRpars,
#    outpath=opth,
#    t0=0, t1=0, # when both times are 0, will use full data
#    itr=2,      # number of iterations used for recon to which registering MR/UTE
#    petopt='ac',# what PET image to use (ac-just attenuation corrected)
#    musrc='ute',# source of mu-map (ute/pct)
#    ute_name='UTE2', # which UTE to use (UTE1/2 shorter/faster)
#    verbose=True,
#)

#> the same as above without any faff though (no alignment)
mu_o = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

In [ ]:
# create histogram
mMRpars['Cnt']['BTP'] = 0
m = nipet.mmrhist(datain, mMRpars, outpath=opth, store=True, use_stored=True)
if False:
    mMRpars['Cnt']['BTP'] = 2  # enable parametric bootstrap
    totCnt = 3e6
    mMRpars['Cnt']['BTPRT'] = totCnt / m['psino'].sum()  # ratio count level relative to the original
    m = nipet.mmrhist(datain, mMRpars, outpath=path.join(opth, 'BTP', '%.3g' % totCnt), store=True)

## Visualisations

In [ ]:
try:  # needs HW mu-maps
    volshow(mu_o['im'] + mu_h['im'], cmaps=['bone'], titles=[r"$\mu$-map"])
except:
    volshow(mu_o['im'], cmaps=['bone'])

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
volshow([m['psino'], m['dsino']],
        titles=["Prompt sinogram (%.3gM)" % (m['psino'].sum() / 1e6),
               "Delayed sinogram (%.3gM)" % (m['dsino'].sum() / 1e6)],
        cmaps=['inferno'] * 2, xlabels=["", "bins"], ylabels=["angles"] * 2, ncols=2);

# Reconstruction

## OSEM

In [ ]:
# built-in default: 14 subsets
recon = nipet.mmrchain(
    datain, mMRpars,
    frames=['timings', [3000, 3600]],
    itr=4,
    histo=m,
    mu_h=mu_h,
    mu_o=mu_o,
    fwhm=2.5,
    outpath=opth,
    fcomment="niftypet-recon",
    store_img=True)

volshow(recon['im'][:, 100:-100, 100:-100], cmaps=['magma']);

## MLEM

### TODO
- check use of `A`, `N`, and `sim`

In [ ]:
## Randoms

r = nipet.randoms(m, mMRpars)[0]
#volshow(r, cmaps=['inferno'])
print("Randoms: %.3g%%" % (r.sum() / m['psino'].sum() * 100))

## Scatter

# One OSEM iteration estimate (implicitly using voxel-driven scatter model)
eim = nipet.mmrchain(datain, mMRpars, mu_h=mu_h, mu_o=mu_o, itr=1, outpath=opth)['im']
# Recalculate scatter
s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), eim, m, r, mMRpars)[0]
#volshow(s, cmaps=['inferno'])
print("Scatter: %.3g%%" % (s.sum() / m['psino'].sum() * 100))

## Attenuation, Normalisation & Sensitivity

A = nipet.frwd_prj(mu_h['im'] + mu_o['im'], mMRpars, attenuation=True)
N = nipet.mmrnorm.get_norm_sino(datain, mMRpars, m)
AN = A * N
#volshow(AN, title='Attenuation & Normalisation Sinogram',
#        cmap='inferno', xlabel='bins', ylabel='angles')
sim = nipet.back_prj(AN, mMRpars)
#volshow(sim, cmaps=['magma'], titles=["Sensitivity"]);

In [ ]:
## MLEM with RM

psf = functools.partial(gaussian_filter, sigma=2.5 / SIGMA2FWHMmm)
recon_mlem = [np.ones_like(sim)]
msk = nipet.img.mmrimg.get_cylinder(mMRpars['Cnt'], rad=29., xo=0., yo=0., unival=1, gpu_dim=False) <= 0.9
sim_inv = 1 / psf(sim)
sim_inv[msk] = 0
rs = r + s
for k in trange(4 * 14, desc="MLEM"):
    fprj = nipet.frwd_prj(psf(recon_mlem[-1]), mMRpars) + rs
    recon_mlem.append(recon_mlem[-1] * sim_inv * psf(nipet.back_prj(m['psino'] / fprj, mMRpars)))

# central slice across iterations
volshow(np.asanyarray(recon_mlem[1::5])[:, :, 100:-100, 100:-100], cmaps=['magma'] * len(recon_mlem[1::5]));

In [ ]:
# central slice across iterations
volshow(np.asanyarray(recon_mlem[1::5])[:, :, 100:-100, 100:-100], cmaps=['magma'] * len(recon_mlem[1::5]));